函数由两部分组成：代码对象（func.\_\_code__）持有字节码和指令元数据，负责执行；函数对象则为上下文提供调用实例，并管理所需的状态数据。

In [6]:
import dis

def test(x, y=10):
    x += y
    print(x,y)
dis.dis(test)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 INPLACE_ADD
              6 STORE_FAST               0 (x)

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (x)
             12 LOAD_FAST                1 (y)
             14 CALL_FUNCTION            2
             16 POP_TOP
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


In [8]:
dis.dis(test.__code__.co_code) # 没有元数据符号的反汇编结果

          0 LOAD_FAST                0 (0)
          2 LOAD_FAST                1 (1)
          4 INPLACE_ADD
          6 STORE_FAST               0 (0)
          8 LOAD_GLOBAL              0 (0)
         10 LOAD_FAST                0 (0)
         12 LOAD_FAST                1 (1)
         14 CALL_FUNCTION            2
         16 POP_TOP
         18 LOAD_CONST               0 (0)
         20 RETURN_VALUE


In [9]:
test.__defaults__

(10,)

In [14]:
test.__defaults__ = (1234,)
test(1)

1235 1234


In [13]:
test.abn = 'sd'
test.__dict__

{'abn': 'sd'}

def实际上是运行期指令，以代码对象为参数，创建函数实例，并在当前上下文中与指定的名字相关联。反汇编操作会在函数实例创建后执行，目标针对\_\_code__，而非创建过程。

In [16]:
def make(n):
    ret = []
    for i in range(n):
        def test():
            print('hello')
        print(id(test), id(test.__code__))
        ret.append(test)
    return ret
make(3)

4400162200 4400053536
4400164512 4400053536
4400194016 4400053536


[<function __main__.make.<locals>.test>,
 <function __main__.make.<locals>.test>,
 <function __main__.make.<locals>.test>]

In [20]:
a = make
a.__name__

'make'

In [22]:
b = lambda x,y: x+y
b.__qualname__

'<lambda>'

In [23]:
def test(a,b,*,c):   # c为无默认值的kwargs，必须显式命名传参
    print(locals())

In [24]:
test(1,2)

TypeError: test() missing 1 required keyword-only argument: 'c'

In [25]:
test(1,2,3)

TypeError: test() takes 2 positional arguments but 3 were given

In [26]:
test(1,2,c=3)

{'c': 3, 'b': 2, 'a': 1}


In [27]:
test(1,2,3,c=4)

TypeError: test() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

- 参数的默认值是在函数对象创建时生成的，保存在\_\_defaults__，为每次调用共享
- kwargs会维持传入顺序
- 收集参数不计入\_\_code__.co_argcount